In [2]:
%pylab inline
import pandas as pd
import numpy as np
import fmt
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


Populating the interactive namespace from numpy and matplotlib


# Homework Set 7

This homework is to price [synthetic CDO](https://en.wikipedia.org/wiki/Synthetic_CDO) using the one factor Gaussian Copula model. 

A synthetic CDO consists of $n$ CDS, the total loss of the portfolio is defned as:

$$ l(t) = \sum_i^n w_i \tilde {\mathbb{1}}_i(t) (1-r_i(t)) $$

where $w_i$ and $r_i(t)$ are the notional weights and recovery rate of the i-th name in the portfolio. The notional weighs sum up to 1: $\sum_i w_i = 1 $. The $ \tilde {\mathbb{1}}_i(t) $ is the default indicator of the i-th name defaulted before time $t$, the default probability is therefore $p_i(t) = \mathbb E[\tilde {\mathbb{1}}_i(t) ]$

For the purpose of this homework, we consider a simplified synthetic CDO that has no coupon payments, therefore the PV of a \$1 notional synthetic CDO tranche with maturity $t$, attachment $a$ and detachment $d$ is:

$$ v(a, d) = \frac{d(t)}{d-a} \min\left((l(t) - a)^+, d-a\right) $$

where $d(t)$ is the discount factor.

The following are the parameters to the synthetic CDO, and a straight forward Monte Carlo pricer:

In [7]:
n = 125
t = 5.
defProbs = 1 - exp(-(np.random.uniform(size=n)*.03)*t)
recovery = 0.4*np.ones(n)
w = 1./n*np.ones(n)
rho = 0.5
discf = .9
npath = 1000

# a list of attachements and detachements, they pair up by elements
attachements = np.array([0, .03, .07, .1, .15, .3])
detachements = np.array([.03, .07, .1, .15, .3, .6])

#portfolio expected loss
el = np.sum(w*defProbs*(1-recovery))
print "portfolio expected loss is ", el

portfolio expected loss is  0.0407459864753


In [8]:
from scipy.stats import norm

class CDO(object) :
    def __init__(self, w, defProbs, recovery, a, d) :
        self.w = w/np.sum(w)
        self.p = defProbs
        self.rec = recovery
        self.rho = rho
        self.a = a
        self.d = d

    def drawDefaultIndicator(self, z, rho) :
        '''return a list of default indicators given common factor z, using one factor Gaussian Copula
        '''
        e = np.random.normal(size=np.shape(self.p))
        x = z*np.sqrt(self.rho) + np.sqrt(1-self.rho)*e
        return np.less(norm.cdf(x), self.p)

    def portfolioLoss(self, defIndicator) :
        '''compute portfolio loss given default indicators'''
        return np.sum(defIndicator*self.w*(1-self.rec))

    def tranchePV(self, portfLoss, discf) :
        '''compute tranche PV from portfolio loss
        Args:
            portfLoss: the total portfolio loss
            discf: discount factor
        Returns:
            tranche PVs'''
        
        sz = self.d - self.a
        return discf/sz*np.minimum(np.maximum(portfLoss - self.a, 0), sz)

    def drawPV(self, z, rho, discf) :
        ''' compute PV and portfolio Loss conditioned on a common factor z'''
        di = self.drawDefaultIndicator(z, rho)
        pfLoss = self.portfolioLoss(di)
        return self.tranchePV(pfLoss, discf), pfLoss
    
    
cdo = CDO(w, defProbs, recovery, attachements, detachements)

In [9]:
## price the tranches using simulation
def simCDO(cdo, rho, disc, paths) :
    zs = np.random.normal(size=[paths])
    pv = np.zeros(np.shape(cdo.a))
    pv2 = np.zeros(np.shape(cdo.d))
    for z in zs:
        thisPV, _ = cdo.drawPV(z, rho, discf)
        pv += thisPV
        pv2 += thisPV*thisPV
        
    v = pv/paths
    var = pv2/paths - v**2
    return pv/paths, np.sqrt(var/paths), var

In [26]:
pv_0, err_0, var_0 = simCDO(cdo, rho, discf, npath)
df = pd.DataFrame(np.array([cdo.a, cdo.d, pv_0, err_0, var_0]), index=['Attach', 'Detach', 'PV', 'MC err', 'Var'])

fmt.displayDF(df, fmt='4g')

,0,1,2,3,4,5
Attach,0,0.03,0.07,0.1,0.15,0.3
Detach,0.03,0.07,0.1,0.15,0.3,0.6
PV,0.4555,0.2208,0.1269,0.08621,0.02878,0.002318
MC err,0.012,0.01139,0.009526,0.007973,0.004222,0.0007348
Var,0.1441,0.1297,0.09075,0.06357,0.01783,0.0005399


## Problem 1

Modify the simCDO function to implement the following variance reduction techniques, and show whether the technique is effective:

For this homework, we only apply the variance reduction in the common market factor $z$, you should not change the random number $e$ that were drew with in the drawDefaultIndicator function, i.e., only modify the simCDO code, re-use but do not modify the CDO class. Unless explicitly mentioned, keep the simulation path the same as the base case above.

1. anti-thetic variate, reduce the number of paths by half to account for the 2x increase in computation
1. importance sampling, shift $z$ by -1
1. sobol sequence
1. stratified sampling: sample $z$ using an equal sized grid

Compute the **variance** reduction factor for each technique, and comment on the effectiveness of these variance reduction techniques.

## Solution:
#### 1.

In [16]:
def simCDO_1(cdo, rho, disc, paths) :
    zs = np.random.normal(size=[paths])
    pv = np.zeros(np.shape(cdo.a))
    pv2 = np.zeros(np.shape(cdo.d))
    for z in zs:
        thisPV1, _ = cdo.drawPV(z, rho, discf)
        thisPV2, _ = cdo.drawPV(-z, rho, discf)
        thisPV = 0.5*(thisPV1 + thisPV2)
        pv += thisPV
        pv2 += thisPV*thisPV
        
    v = pv/paths
    var = pv2/paths - v**2
    return pv/paths, np.sqrt(var/paths), var

In [27]:
pv_1, err_1, var_1 = simCDO_1(cdo, rho, discf, npath/2)
df = pd.DataFrame(np.array([cdo.a, cdo.d, pv_1, err_1, var_1]), index=['Attach', 'Detach', 'PV', 'MC err', 'Var'])

print "\nAnti-thetic: \n"
fmt.displayDF(df, fmt='4g')
print "\nvariance reduction factor:", var_0/var_1


Anti-thetic: 



,0,1,2,3,4,5
Attach,0,0.03,0.07,0.1,0.15,0.3
Detach,0.03,0.07,0.1,0.15,0.3,0.6
PV,0.4541,0.2319,0.1407,0.08682,0.03569,0.002246
MC err,0.004688,0.008893,0.008855,0.007453,0.004667,0.0007482
Var,0.01099,0.03954,0.03921,0.02777,0.01089,0.0002799



variance reduction factor: [ 13.11580478   3.2803555    2.31468522   2.28866857   1.63671808
   1.92905716]


We can see that anti-thetic works well on this problem, the variance reduction is significant.

#### 2.

In [18]:
def simCDO_2(cdo, rho, disc, paths) :
    #zs = np.random.normal(size=[paths])
    u = -1.
    xs_q = np.random.normal(size=[paths]) # Q sample
    xs_p = xs_q + u # P sample
    qs = 1./paths*np.ones(paths)
    zs = np.exp(-u*xs_p + .5*u*u) # R-N derivative
    #ps = qs*zs
    #ps = ps/sum(ps)  # normalize
    
    pv = np.zeros(np.shape(cdo.a))
    pv2 = np.zeros(np.shape(cdo.d))
    for i in xrange(len(xs_p)):
        thisPV, _ = cdo.drawPV(xs_p[i], rho, discf)
        thisPV *= zs[i]
        pv += thisPV
        pv2 += thisPV*thisPV
        
    v = pv/paths
    var = pv2/paths - v**2
    return pv/paths, np.sqrt(var/paths), var

In [28]:
nBatches = 20

PV_2, ERR_2, VAR_2 = [], [], []
for i in xrange(nBatches):
    pv_2, err_2, var_2 = simCDO_2(cdo, rho, discf, npath)
    PV_2.append(pv_2)
    ERR_2.append(err_2)
    VAR_2.append(var_2)
df = pd.DataFrame(np.array([cdo.a, cdo.d, np.mean(PV_2, 0), np.mean(ERR_2, 0), np.mean(VAR_2, 0)]),\
                  index=['Attach', 'Detach', 'PV', 'MC err', 'Var'])

print "\nImportance sampling:\n"
fmt.displayDF(df, fmt='4g')
print "\nvariance reduction factor:", var_0/var_2


Importance sampling:



,0,1,2,3,4,5
Attach,0,0.03,0.07,0.1,0.15,0.3
Detach,0.03,0.07,0.1,0.15,0.3,0.6
PV,0.4473,0.2304,0.1409,0.08814,0.03137,0.002521
MC err,0.009466,0.006408,0.005057,0.003673,0.001658,0.000259
Var,0.08965,0.04107,0.02558,0.01349,0.00275,6.732e-05



variance reduction factor: [ 1.59290947  3.08784805  3.45572554  4.59835026  6.78493891  8.79514383]


We can see that importance sampling also gains significant variance reduction.

#### 3.

In [20]:
import sobol
def simCDO_3(cdo, rho, disc, paths) :
    e = sobol.sobol_seq.i4_sobol_generate(1, 1000, 2)
    zs = norm.ppf(e[0])
    pv = np.zeros(np.shape(cdo.a))
    pv2 = np.zeros(np.shape(cdo.d))
    for z in zs:
        thisPV, _ = cdo.drawPV(z, rho, discf)
        pv += thisPV
        pv2 += thisPV*thisPV
        
    v = pv/paths
    var = pv2/paths - v**2
    return pv/paths, np.sqrt(var/paths), var

In [29]:
nBatches = 20

PV_3, ERR_3, VAR_3 = [], [], []
for i in xrange(nBatches):
    pv_3, err_3, var_3 = simCDO_3(cdo, rho, discf, npath)
    PV_3.append(pv_3)
    ERR_3.append(err_3)
    VAR_3.append(var_3)
df = pd.DataFrame(np.array([cdo.a, cdo.d, np.mean(PV_3, 0), np.mean(ERR_3, 0), np.mean(VAR_3, 0)]),\
                  index=['Attach', 'Detach', 'PV', 'MC err', 'Var'])

print "\nSobol sequence:\n"
fmt.displayDF(df, fmt='4g')
print "\nvariance reduction factor:", var_0/var_3


Sobol sequence:



,0,1,2,3,4,5
Attach,0,0.03,0.07,0.1,0.15,0.3
Detach,0.03,0.07,0.1,0.15,0.3,0.6
PV,0.4494,0.229,0.1385,0.08683,0.03087,0.001817
MC err,0.01223,0.01153,0.009857,0.007941,0.00436,0.0006616
Var,0.1496,0.1329,0.09717,0.06308,0.01902,0.0004419



variance reduction factor: [ 0.97144033  0.98250738  0.95518904  1.01268032  1.017612    1.17981172]


For this particular problem, the sobol sequence shows no significant improvement.

#### 4.

In [22]:
def stratify(u, bs, shuffle) :
    b = len(bs)
    r = len(u)/b + 1
    sb = []    
    for i in range(r) :
        if shuffle :
            np.random.shuffle(bs)
        sb = sb + bs.tolist()
            
    return [1.*(i + x)/b for x, i in zip(u, sb)]

In [23]:
def simCDO_4(cdo, rho, disc, paths) :
    bs = np.arange(100)
    u = np.random.uniform(size=[paths])
    v = stratify(u, bs, False)
    zs = norm.ppf(v)
    pv = np.zeros(np.shape(cdo.a))
    pv2 = np.zeros(np.shape(cdo.d))
    for z in zs:
        thisPV, _ = cdo.drawPV(z, rho, discf)
        pv += thisPV
        pv2 += thisPV*thisPV
        
    v = pv/paths
    var = pv2/paths - v**2
    return pv/paths, np.sqrt(var/paths), var

In [30]:
nBatches = 20

PV_4, ERR_4, VAR_4 = [], [], []
for i in xrange(nBatches):
    pv_4, err_4, var_4 = simCDO_4(cdo, rho, discf, npath)
    PV_4.append(pv_4)
    ERR_4.append(err_4)
    VAR_4.append(var_4)
df = pd.DataFrame(np.array([cdo.a, cdo.d, np.mean(PV_4, 0), np.mean(ERR_4, 0), np.mean(VAR_4, 0)]),\
                  index=['Attach', 'Detach', 'PV', 'MC err', 'Var'])

print "\nStratified sampling:\n"
fmt.displayDF(df, fmt='4g')
print "\nvariance reduction factor:", var_0/var_4


Stratified sampling:



,0,1,2,3,4,5
Attach,0,0.03,0.07,0.1,0.15,0.3
Detach,0.03,0.07,0.1,0.15,0.3,0.6
PV,0.4501,0.2317,0.1402,0.08666,0.03077,0.002308
MC err,0.01223,0.01159,0.009888,0.007931,0.004375,0.0008371
Var,0.1496,0.1343,0.09779,0.06293,0.01915,0.0007481



variance reduction factor: [ 0.96682085  0.97593273  0.92710816  0.99392895  0.97583747  3.44262277]


Similar to sobol sequence, stratified sampling also shows no significant improvement.

## (Extra Credit) Problem 2

Consider a control variate for the problem above. The large pool model assumes that the portfolio is a large homogeneous pool, using the average default rate: $\bar p = \frac{1}{n}\sum_i p_i$. Then the portfolio loss conditioned on market factor $z$ under the large pool model is a determinsitic scalar:

$$ l(z) = (1-r)\Phi\left(\frac{\Phi^{-1}(\bar p) - \sqrt \rho z}{\sqrt{1-\rho}}\right)$$

where $r$ is the constant recovery of all names. $\Phi()$ is the normal CDF function; $\Phi^{-1}()$ is its inverse. The tranche PVs can then be computed from the $l(z)$.

Please investigate if the large pool model can be used as an effective control variate. Does it work better for some tranches?

Hint: to answer this question, you only need to compute the correlation between the actual and control variates. 